In [ ]:
# this notebook will show case how to use the helper function to train decision transformer agent with the dataset

# define the dataset path
dataset_path = "offline_stock_trade_data"

In [ ]:
# first we need to calculate the mean and std of each feature in the dataset
from curatedataset import calc_meanstd_datasets
dataset_meanstd = calc_meanstd_datasets(dataset_path, ["positive", "negative", "neutral"])


In [ ]:
# then we can train the agent using the full_training_run helper function
from train_decision_transformer import full_training_run
import torch

# set device to cuda if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
trained_model, trained_params = full_training_run(
    dataset_path, device = device, force_normalize=dataset_meanstd, n_epochs=40)

In [ ]:
# save the trained model
from train_decision_transformer import save_model

model_name = 'nasdaq_dow_news_meanstd'
model_path = 'trained_nasdaq_dow_decision_transformer'
#save_model(trained_model, trained_params, model_name, model_path)

In [ ]:
# we first need to see what are rewards distribution in the dataset

from curatedataset import eval_reward_datasets
mean_rewards = eval_reward_datasets(dataset_path)

In [ ]:
# we then can use the reward as RTG for agent to trade
# rtg = mean_rewards['reward_75'][0]
rtg = 28.7

In [ ]:
# we will then create the gym environment to test run the agent
# the gym environment will be normalized using the same mean_std as what is used to train the model

import json
import os
from TradingEnvClass import MeanStdObject

# get file end with params.json
json_files = [f for f in os.listdir(model_path) if f.endswith('params.json')]
parameter_path = os.path.join(model_path, json_files[0])
with open(parameter_path, 'r') as f:
    normalize = json.load(f)["normalize"]

mean_std_dict = {key: MeanStdObject(mean = value['mean'], std=value['std']) 
                 for key, value in normalize.items()}


In [ ]:
from curatedataset import makegymenv, run_env
test_tic = ['AAPL', 'MSFT', 'GOOGL', 'TSLA', 'AMZN', 'META', 'NVDA', 'INTC', 'AMD']

# TODO test different start date within the dataset 
start_date = '2020-06-01'
num_days = 365
interval = '1d'
indicators = ["Volume", "volume_cmf", "trend_macd", "momentum_rsi", "momentum_stoch_rsi", "trend_sma_fast"]
init_balance = 20000
test_env_list_norm = []
test_env_list = []
#data = get_stock_data_yf_between_with_indicators(stock_name, '2019-01-01', '2020-02-01', interval, ['all'])
"""
for stock_name in test_tic:
    env, obs_space, act_space, col, data = makegymenv(stock_name, start_date, num_days, interval, indicators=indicators, normalize=mean_std_dict, init_balance=init_balance)
    test_env_list_norm.append(env)
"""
for stock_name in test_tic:
    env, obs_space, act_space, col, data = makegymenv(stock_name, start_date, num_days, interval, indicators=indicators, init_balance=init_balance)
    test_env_list.append(env)


In [ ]:
# then create an agent from model weight to interact with the environment
from get_agent import Agent, TradingAlgorithm
import torch
import fnmatch

# get model parameter's path under model path by checking for _params.json
f = os.listdir(model_path)
matching_files = fnmatch.filter(f, model_name+'*_params.json')
# get the file that store the model information
parameter_path = os.path.join(model_path, matching_files[0])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
"""
# create agent (use the first env as sample env)
DecisionTransformer = Agent(test_env_list_norm[0], 'transformer', 
                            rtg_target=rtg, rtg_scale=0.7, model_path=parameter_path, device=device)
"""
# create a random agent for comparison
RandomAgent = Agent(test_env_list[0], 'random')
# create a momentum algo agent for comparison
momentumAlgo = TradingAlgorithm(algo_type='momentum_stoch_rsi', 
                                indicator_column = col.index('momentum_stoch_rsi'), amount_range = [0.05, 0.55])
momentumAgent = Agent(test_env_list[0], 'algo', algo=momentumAlgo)
# create a sentiment algo agent for comparison
sentimentAlgo = TradingAlgorithm(algo_type='sentiment_react', 
                                 indicator_column = [col.index('positive'), col.index('negative'), col.index('neutral')], 
                                 amount_range = [0.05, 0.55])
sentimentAgent = Agent(test_env_list[0], 'algo', algo=sentimentAlgo)

In [ ]:
# run and save the interaction data
from curatedataset import save_data
"""
for env, stock_name in zip(test_env_list_norm, test_tic):
    trade_data = run_env(DecisionTransformer, stock_name, env, 3, start_date, normalize_param = mean_std_dict)
    file_name = os.path.join(model_path, f'trained_transformer_{stock_name}_{start_date}_{num_days}_{interval}_test.json')
    save_data(trade_data, file_name)
"""
# also run the other agent for comparison
test_compare_directory = 'test_compare'
# check if the directory exist
if not os.path.exists(test_compare_directory):
    os.makedirs(test_compare_directory)
for env, stock_name in zip(test_env_list, test_tic):
    trade_data = run_env(RandomAgent, stock_name, env, 3, start_date)
    file_name = os.path.join(test_compare_directory, f'random_agent_{stock_name}_{start_date}_{num_days}_{interval}_test.json')
    save_data(trade_data, file_name)
    trade_data = run_env(momentumAgent, stock_name, env, 3, start_date)
    file_name = os.path.join(test_compare_directory, f'momentum_agent_{stock_name}_{start_date}_{num_days}_{interval}_test.json')
    save_data(trade_data, file_name)
    trade_data = run_env(sentimentAgent, stock_name, env, 3, start_date)
    file_name = os.path.join(test_compare_directory, f'sentiment_agent_{stock_name}_{start_date}_{num_days}_{interval}_test.json')
    save_data(trade_data, file_name)


In [ ]:
# we then can use the evaluation helper to plot the stock price movement and trading action.
# this is for the decision transformer agent
from curatedataset import evaluate_dataset
import os

json_files = [f for f in os.listdir(model_path) if f.endswith('1d_test.json')]
# get the index of list from tic
keywords = ['NVDA']
# get the json file that contain the keyword
ticindex = [i for i, item in enumerate(json_files) if all(k in item for k in keywords)][0]
eval_data = os.path.join(model_path,json_files[ticindex])
print(eval_data)
evaluate_dataset(eval_data)

In [ ]:
# we then can use the evaluation helper to plot the stock price movement and trading action.
# this is for the other agent

json_files = [f for f in os.listdir(test_compare_directory) if f.endswith('1d_test.json')]
# get the index of list from tic
keywords = ['momentum', 'NVDA']
# get the json file that contain the keyword
ticindex = [i for i, item in enumerate(json_files) if all(k in item for k in keywords)][0]
eval_data = os.path.join(test_compare_directory,json_files[ticindex])
print(eval_data)
evaluate_dataset(eval_data)